In [ ]:
import numpy as np
import keras.backend as K
from machinedesign.viz import grid_of_images_default
from skimage.io import imsave
from keras.models import Model
from skimage.transform import resize
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from machinedesign.autoencoder.interface import load
from sklearn.neighbors import KDTree
from machinedesign.viz import horiz_merge
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
folder = 'mnist'
data = np.load('../results/{}/gen/generated.npz'.format(folder))
model = load('../results/{}'.format(folder))

In [ ]:
#mnist
inp = model.layers[0].input
hid_layer = model.get_layer('activation_3')
hid = hid_layer.output
out = model.layers[-1].output
enc = K.function([inp], hid)
dec = K.function([hid], out)

In [ ]:
data = np.load('../../data/digits.npz')
Xtrue = data['X'] / 255.
Xtrue = Xtrue[0:1000]
htrue = enc([Xtrue])
#htrue = htrue > 0
#htrue = htrue.max(axis=(2, 3))
htrue = htrue.reshape((htrue.shape[0], -1))

In [ ]:
np.random.seed(42)
x = np.random.uniform(size=(1000, 1, 28, 28))
hlist = []
for i in range(100):s
    h = enc([x])
    hlist.append(h)
    x = dec([h])
hfake = h
hfake = hfake.reshape((hfake.shape[0], -1))
Xfake  = x
x = grid_of_images_default(x)
imsave('out.png', x)

In [ ]:
tree = KDTree(htrue)
dist, ind = tree.query(hfake, k=3)
x = Xtrue[ind[:, 0]]
a = grid_of_images_default(x)

x = Xfake
b = grid_of_images_default(x)

x = horiz_merge(a, b)
imsave('out.png', x)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
H = np.concatenate( (htrue.reshape((htrue.shape[0], -1)), hfake.reshape((hfake.shape[0], -1))), axis=0)
X = np.concatenate( (Xtrue, Xfake), axis=0)
Y = np.array([1] * len(htrue) + [0] * len(hfake))

print(X.shape, Y.shape, H.shape)
ind = np.arange(len(H))
np.random.shuffle(ind)
X = X[ind]
H = H[ind]

Y = Y[ind]

clf = make_pipeline(PCA(n_components=100), LogisticRegression())
#clf = RandomForestClassifier(max_leaf_nodes=10, n_estimators=100)
#clf = SVC()
clf.fit(H[0:900], Y[0:900])

In [ ]:
(clf.predict(H[900:])==Y[900:]).mean()

In [ ]:
x = X[(Y==0) * (clf.predict(H)==1)]
print(x.shape)
im = grid_of_images_default(x)
imsave('out.png', im)